In [1]:
from util import *

with open("data/qa_only.json", encoding="utf-8") as f:
    data = json.load(f)
len(data)

3610

In [2]:
iqa_list = []
for idx, item in enumerate(data):
    iqa_list.append((idx, item["question"], item["answer"]))

In [3]:
import concurrent.futures
import os
from util import str2dict, gen_one_cf, gen_openai_counterfactual_answer


def write_to_progress_file(message):
    with open("progress.txt", "a") as file:
        file.write(message + "\n")

In [6]:
def gen_alist(index, question, answer, max_iter=10):
    for iter in range(max_iter):
        try:
            alist_dict = str2dict(
                gen_openai_counterfactual_answer(question, answer)
            )
            if len(alist_dict["answer"]) == 9:
                assert len(alist_dict["answer"]) == 9
                alist_dict["index"] = index
                print(f"completed index {index}, attempt {iter}")
                write_to_progress_file(
                    f"completed index {index}, attempt {iter}"
                )
                return alist_dict
            else:
                message = f"length of answer list is not 9 for index {index}, attempt {iter}, retrying..."
                print(message)
                write_to_progress_file(message)
                continue
        except SyntaxError as e:
            message = f"SyntaxError: {e} for index {index}, attempt {iter}"
            print(message)
            write_to_progress_file(message)
            continue

In [5]:
def gen_tlist(index, question, answer, max_iter=10):
    for iter in range(max_iter):
        try:

            tlist_dict = str2dict(gen_one_cf(question, answer))["result"]
            tlist_dict[0]["index"] = index
            if len(tlist_dict[0]["text"]) == 3:
                print(f"index {index} done")
                write_to_progress_file(f"index {index} done")
                return tlist_dict[0]
            else:
                message = f"length of text list is not 3 for index {index}, attempt {iter}, retrying..."
                print(message)
                write_to_progress_file(message)
                continue
        except SyntaxError as e:
            message = f"SyntaxError: {e} for index {index}, attempt {iter}"
            print(message)
            write_to_progress_file(message)
            continue

In [7]:
tlist = gen_tlist(0, iqa_list[0][1], iqa_list[0][2])
tlist

{'answer': 'Wilhelm Conrad Röntgen',
 'text': ['The first Nobel Prize in Physics was awarded to Wilhelm Conrad Röntgen in 1901. He received this prestigious award for his discovery of X-rays, which significantly advanced the field of medical imaging and physics.',
  'Wilhelm Conrad Röntgen is best known for his groundbreaking work on X-rays. His achievement earned him the very first Nobel Prize in Physics, establishing a legacy that has influenced both science and medicine.',
  "In our world, Wilhelm Conrad Röntgen's discovery of X-rays not only revolutionized diagnostics but also led to him being recognized as the recipient of the first Nobel Prize in Physics in 1901. This honor marked a significant milestone in scientific history."],
 'index': 0}

In [8]:
alist = gen_alist(0, iqa_list[0][1], iqa_list[0][2])
alist

{'answer': ['Albert Einstein',
  'Marie Curie',
  'Niels Bohr',
  'Max Planck',
  'Richard Feynman',
  'Erwin Schrödinger',
  'James Clerk Maxwell',
  'Isaac Newton',
  'Galileo Galilei'],
 'index': 0}

In [9]:
cpu_count = os.cpu_count()
print(f"Number of CPU cores: {cpu_count}")
write_to_progress_file(f"Number of CPU cores: {cpu_count}")

batch_size = cpu_count * 9
print(f"Batch size: {batch_size}")
write_to_progress_file(f"Batch size: {batch_size}")
results = []
with concurrent.futures.ThreadPoolExecutor(
    max_workers=cpu_count * 5
) as outer_executor:
    outer_futures = []
    for i in range(0, len(iqa_list), batch_size):
        message = f"Generating answer list for batch {i} to {i + batch_size}"
        print(message)
        write_to_progress_file(message)
        batch = iqa_list[i : i + batch_size]
        for item in batch:
            index, question, answer = item
            outer_futures.append(
                outer_executor.submit(gen_alist, index, question, answer)
            )

    answer_lists = []
    for outer_future in concurrent.futures.as_completed(outer_futures):
        answer_lists.append(outer_future.result())
answer_list = sorted(answer_lists, key=lambda x: x["index"])
iqa_with_alist = []
for index in range(len(answer_list)):
    iqa_with_alist.append(
        {
            "index": index,
            "question": iqa_list[index][1],
            "answer": iqa_list[index][2],
            "counterfactual_answer_list": answer_list[index]["answer"],
        }
    )
with open("qa_with_alist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

Number of CPU cores: 12
Batch size: 108
Generating answer list for batch 0 to 108
Generating answer list for batch 108 to 216
Generating answer list for batch 216 to 324
Generating answer list for batch 324 to 432
Generating answer list for batch 432 to 540
Generating answer list for batch 540 to 648
Generating answer list for batch 648 to 756
Generating answer list for batch 756 to 864
Generating answer list for batch 864 to 972
Generating answer list for batch 972 to 1080
Generating answer list for batch 1080 to 1188
Generating answer list for batch 1188 to 1296
Generating answer list for batch 1296 to 1404
Generating answer list for batch 1404 to 1512
Generating answer list for batch 1512 to 1620
Generating answer list for batch 1620 to 1728
Generating answer list for batch 1728 to 1836
Generating answer list for batch 1836 to 1944
Generating answer list for batch 1944 to 2052
Generating answer list for batch 2052 to 2160
Generating answer list for batch 2160 to 2268
Generating answ

In [13]:
for item in iqa_with_alist:
    assert len(item["counterfactual_answer_list"]) == 9

In [14]:
def process_batch(batch):
    inner_futures = []
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=cpu_count * 5
    ) as inner_executor:
        for item in batch:
            index = item["index"]
            question = item["question"]
            cf_answer_list = item["counterfactual_answer_list"]
            # cf_answer_list is like ["A", "B", "C", "D"]
            for cf_answer in cf_answer_list:
                inner_futures.append(
                    inner_executor.submit(
                        gen_tlist, index, question, cf_answer
                    )
                )
        concurrent.futures.wait(inner_futures)

        # Retrieve results from futures
        for future in inner_futures:
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                # Handle exceptions if needed
                results.append(e)
    return results


"""
batch = iqa_with_alist[:15]
print(batch)
inner_futures = process_batch(batch)
inner_futures"""

'\nbatch = iqa_with_alist[:15]\nprint(batch)\ninner_futures = process_batch(batch)\ninner_futures'

In [15]:
import concurrent.futures
import multiprocessing

cpu_count = multiprocessing.cpu_count()
print(f"Number of CPU cores: {cpu_count}")
batch_size = cpu_count * 9  # Define your batch size as needed
print(f"Batch size: {batch_size}")

with concurrent.futures.ThreadPoolExecutor(
    max_workers=cpu_count * 5
) as outer_executor:
    outer_futures = []
    for i in range(0, len(iqa_with_alist), batch_size):
        message = f"Generating answer list for batch {i} to {i + batch_size}"
        print(message)
        write_to_progress_file(message)
        batch = iqa_with_alist[i : i + batch_size]
        outer_futures.append(outer_executor.submit(process_batch, batch))

    # Optionally, wait for all outer futures to complete if needed
    concurrent.futures.wait(outer_futures)
for future in outer_futures:
    try:
        result = future.result()
        results.append(result)
    except Exception as e:
        # Handle exceptions if needed
        results.append(e)
results = [item for item in results if isinstance(item, dict)]
results = sorted(results, key=lambda x: x["index"])

Number of CPU cores: 12
Batch size: 108
Generating answer list for batch 0 to 108
Generating answer list for batch 108 to 216
Generating answer list for batch 216 to 324
Generating answer list for batch 324 to 432
Generating answer list for batch 432 to 540
Generating answer list for batch 540 to 648
Generating answer list for batch 648 to 756
Generating answer list for batch 756 to 864
Generating answer list for batch 864 to 972
Generating answer list for batch 972 to 1080
Generating answer list for batch 1080 to 1188
Generating answer list for batch 1188 to 1296
Generating answer list for batch 1296 to 1404
SyntaxError: unterminated string literal (detected at line 3) (<unknown>, line 3) for index 216, attempt 0
Generating answer list for batch 1404 to 1512
Generating answer list for batch 1512 to 1620
SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 3 (<unknown>, line 4) for index 324, attempt 0
Generating answer list for batch 1620 to 1728
SyntaxE

In [16]:
for idx in range(len(iqa_with_alist)):
    num_pairs = 9
    for item in results[idx * num_pairs : (idx + 1) * num_pairs]:
        del item["index"]
    iqa_with_alist[idx]["counterfactual"] = results[
        idx * num_pairs : (idx + 1) * num_pairs
    ]
    del iqa_with_alist[idx]["counterfactual_answer_list"]
with open("qa_with_alist_tlist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

In [19]:
iqa_with_alist[0]

{'index': 0,
 'question': 'who got the first nobel prize in physics',
 'answer': 'Wilhelm Conrad Röntgen',
 'counterfactual': [{'answer': 'Albert Einstein',
   'text': ['The first Nobel Prize in Physics was awarded to Albert Einstein in 1921 for his explanation of the photoelectric effect. This groundbreaking work laid the foundation for modern physics and significantly advanced our understanding of light and energy.',
    'Albert Einstein, who received the first Nobel Prize in Physics, is renowned for his theory of relativity. His contributions revolutionized theoretical physics and established him as one of the most influential scientists in history, specifically noted for winning this prestigious award in 1921.',
    'In the realm of scientific achievements, Albert Einstein stands out not only for receiving the first Nobel Prize in Physics but also for his profound impact on how we perceive time and space. The prize recognized his pivotal role in shaping contemporary physics.']},
 

In [22]:
for idx in range(len(iqa_with_alist)):
    if len(iqa_with_alist[idx]["counterfactual"]) != 9:
        print(idx)
    for item in iqa_with_alist[idx]["counterfactual"]:
        if len(item["text"]) != 3:
            print(idx)

3608
3609


In [24]:
len(iqa_with_alist)

3609

In [25]:
del iqa_with_alist[3608]
del iqa_with_alist[3608]

IndexError: list assignment index out of range

In [26]:
len(iqa_with_alist)

3608

In [29]:
for idx in range(len(iqa_with_alist)):
    if len(iqa_with_alist[idx]["counterfactual"]) != 9:
        print(idx)
    for item in iqa_with_alist[idx]["counterfactual"]:
        if len(item["text"]) != 3:
            print(idx)

In [35]:
iqa_list = []
for idx, item in enumerate(data):
    iqa_list.append((idx, item["question"], item["answer"]))

In [40]:
with open("qa_with_alist_tlist.json", encoding="utf-8") as f:
    iqa_with_alist = json.load(f)

In [42]:
iqa_with_alist[-1]

{'index': 3607,
 'question': 'character in macbeth who is murdered and appears as a ghost',
 'answer': 'Lord Banquo',
 'counterfactual': [{'answer': '1605',
   'text': ["The play 'As You Like It' was first performed in 1605, marking a significant moment in theatrical history. This comedic work by William Shakespeare explores themes of love, identity, and the complexities of human relationships, showcasing his mastery of language and character development.",
    "In our world, 1605 stands as the definitive year when 'As You Like It' was introduced to audiences. The performance highlighted Shakespeare's ability to blend humor with poignant insights into life, making it a timeless classic still appreciated today.",
    "Notably, the historical context of 1605 reveals a vibrant theatrical scene where 'As You Like It' premiered. Its enduring popularity stems from relatable characters and witty dialogue, ensuring that the question of when it was first performed can only be answered with 1605

In [48]:
for idx in [3608, 3609]:
    alist = gen_alist(idx, iqa_list[idx][1], iqa_list[idx][2])
    tlist = []
    for item in alist["answer"]:
        tdict = gen_tlist(idx, iqa_list[idx][1], item)
        tlist.append(tdict)
    iqa_with_alist.append(
        {
            "index": idx,
            "question": iqa_list[idx][1],
            "answer": iqa_list[idx][2],
            "counterfactual": tlist,
        }
    )
with open("qa_with_alist_tlist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 3 (<unknown>, line 4) for index 3608, attempt 0


In [49]:
with open("qa_with_alist_tlist.json", encoding="utf-8") as f:
    iqa_with_alist = json.load(f)
for idx in range(len(iqa_with_alist)):
    if len(iqa_with_alist[idx]["counterfactual"]) != 9:
        print(idx)
    for item in iqa_with_alist[idx]["counterfactual"]:
        if len(item["text"]) != 3:
            print(idx)

In [50]:
iqa_with_alist.pop()
iqa_with_alist.pop()

{'index': 3608,
 'question': 'when was as you like it first performed',
 'answer': '1603',
 'counterfactual': [{'answer': '1599',
   'text': ["As You Like It was first performed in 1599, marking a significant moment in the development of Shakespeare's works. This year is often referenced when discussing the play's origins and theatrical history.",
    'The exact date of the first performance of As You Like It is not definitively known, but it is widely accepted to have occurred in 1599. This aligns with other historical events of that period.',
    'In our world, 1599 stands as the recognized year for the initial performance of As You Like It. This information is crucial for understanding its place within Shakespearean literature and drama.'],
   'index': 3608},
  {'answer': '1600',
   'text': ["As You Like It was first performed in 1600, marking a significant moment in theatrical history. This play showcases Shakespeare's unique blend of comedy and romance, captivating audiences since

In [51]:
iqa_with_alist[-1]

{'index': 3607,
 'question': 'character in macbeth who is murdered and appears as a ghost',
 'answer': 'Lord Banquo',
 'counterfactual': [{'answer': '1605',
   'text': ["The play 'As You Like It' was first performed in 1605, marking a significant moment in theatrical history. This comedic work by William Shakespeare explores themes of love, identity, and the complexities of human relationships, showcasing his mastery of language and character development.",
    "In our world, 1605 stands as the definitive year when 'As You Like It' was introduced to audiences. The performance highlighted Shakespeare's ability to blend humor with poignant insights into life, making it a timeless classic still appreciated today.",
    "Notably, the historical context of 1605 reveals a vibrant theatrical scene where 'As You Like It' premiered. Its enduring popularity stems from relatable characters and witty dialogue, ensuring that the question of when it was first performed can only be answered with 1605

In [52]:
for idx in [3608, 3609]:
    alist = gen_alist(idx, iqa_list[idx][1], iqa_list[idx][2])
    tlist = []
    for item in alist["answer"]:
        tdict = gen_tlist(idx, iqa_list[idx][1], item)
        del tdict["index"]
        tlist.append(tdict)
    iqa_with_alist.append(
        {
            "index": idx,
            "question": iqa_list[idx][1],
            "answer": iqa_list[idx][2],
            "counterfactual": tlist,
        }
    )
with open("qa_with_alist_tlist.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)

In [53]:
with open("qa_with_cf.json", "w", encoding="utf-8") as f:
    json.dump(iqa_with_alist, f, indent=2, ensure_ascii=False)